# Challenge 2
## Sort colors by weight

The marbles are color coded by weight. But you don't know which color corresponds to what weight. Put all marbles in the leftmost column, sorted by weight with the heaviest at the bottom, lightest on top.

The two rightmost columns are equipped with a balance scale <span style="font-size:32px;">⚖️</span>. It can tell you which column contains the heaviest marble(s). Use the provided API to read the scales output.

In [ ]:
import asyncio
%pip install ballsort
from ballsort.ballsort_display_utils import open_bs_window
open_bs_window()

In [ ]:
from control_factory import get_ch2_control_sim
bc = get_ch2_control_sim(delay_multiplier=0.6)

In [ ]:
from ch2_scenario import Ch2Scenario
await bc.set_scenario(Ch2Scenario())

In [ ]:
from ball_control import BallControl
from state_update_model import StatePosition

In [ ]:
async def move_ball(bc: BallControl, src: StatePosition, dest: StatePosition):    
    rel_x = src.x - bc.get_position().x
    rel_y = src.y - bc.get_position().y
    await asyncio.gather(
        bc.move_horizontally(rel_x),
        bc.move_vertically(rel_y),
        bc.open_claw())
    await bc.close_claw()
    
    rel_x = dest.x - bc.get_position().x
    rel_y = dest.y - bc.get_position().y
    await asyncio.gather(
        bc.move_horizontally(rel_x),
        bc.move_vertically(rel_y))
    await bc.open_claw()

In [ ]:
def scale_output_positions(scale_output: int) -> tuple[int, int]:
    if scale_output < 0:
        return 2,3
    return 3,2

In [ ]:
async def challenge2_solution():    
    # populate scale
    await move_ball(bc=bc, src=StatePosition(x=1, y=2), dest=StatePosition(x=2, y=4))
    await move_ball(bc=bc, src=StatePosition(x=1, y=3), dest=StatePosition(x=3, y=4))
    x1_heavy, x1_light = scale_output_positions(await bc.read_scales())

    # move heaviest to leftmost column bottom
    await move_ball(bc=bc, src=StatePosition(x=x1_heavy, y=4), dest=StatePosition(x=0, y=4))

    # move the last one to vacant scale column
    await move_ball(bc=bc, src=StatePosition(x=1, y=4), dest=StatePosition(x=x1_heavy, y=4))

    x2_heavy, x2_light = scale_output_positions(await bc.read_scales())
    
    if x2_heavy == x1_light:
        await move_ball(bc=bc, src=StatePosition(x=x2_heavy, y=4), dest=StatePosition(x=0, y=3))
        await move_ball(bc=bc, src=StatePosition(x=x2_light, y=4), dest=StatePosition(x=0, y=2))
        return

    assert(x1_heavy == x2_heavy)
    assert(x1_light == x2_light)

    # temporarily move the light one to column 1
    await move_ball(bc=bc, src=StatePosition(x=x2_light, y=4), dest=StatePosition(x=1, y=4))

    # col0 -> vacant scale position
    await move_ball(bc=bc, src=StatePosition(x=0, y=4), dest=StatePosition(x=x2_light, y=4))

    x3_heavy, x3_light = scale_output_positions(await bc.read_scales())
    await move_ball(bc=bc, src=StatePosition(x=x3_heavy, y=4), dest=StatePosition(x=0, y=4))
    await move_ball(bc=bc, src=StatePosition(x=x3_light, y=4), dest=StatePosition(x=0, y=3))
    assert(not bc.is_in_goal_state())
    await move_ball(bc=bc, src=StatePosition(x=1, y=4), dest=StatePosition(x=0, y=2))
    assert(bc.is_in_goal_state())
    

In [ ]:
await challenge2_solution()